<a href="https://colab.research.google.com/github/nagusubra/Solar_panel_dust_detection/blob/main/Final_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install libraries and modules

In [1]:
!pip install -q tensorflow-model-optimization
!pip install openpyxl
!pip install xlsxwriter

import xlsxwriter
import openpyxl
from openpyxl import Workbook
from openpyxl.drawing.image import Image

import tempfile
from tensorflow import keras
import tensorflow_model_optimization as tfmot

from tqdm import tqdm_notebook as tqdm
import os
import time
import pickle
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 46.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 13.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import binary_crossentropy
import tensorflow_datasets as tfds
import zipfile

In [3]:
# mounting google drive (if you are using Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Evaluation function

In [4]:
# Evaluate Model Size
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  return os.path.getsize(zipped_file)


def evaluate_model(model_path, model_info, val_dataset):

  # Evaluate test accuracy and test loss
  model = tf.keras.models.load_model(model_path)
  test_loss, test_acc = model.evaluate(val_dataset, verbose=0)

  # Evaluate Model Size
  model_size = get_gzipped_model_size(model_path)

  # Evaluate Inference Time
  startTime = time.time()
  prediction = model.predict(val_dataset)
  executionTime = (time.time() - startTime)/len(val_dataset)

  # Print
  print('\nModel Accuracy:', test_acc*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Model Information": model_info,
                      "Accuracy": str(test_acc*100) + " %",
                      "Loss": str(test_loss*100) + " %",
                      "Model Size": str(model_size) + " bytes",
                      "Inference Time": str(executionTime) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()


  return test_acc, model_size, executionTime, evulation_df


def evaluate_model_without_saving_stats(model, model_path, model_info, val_dataset):

  # Evaluate test accuracy and test loss
  # model = tf.keras.models.load_model(model_path)
  test_loss, test_acc = model.evaluate(val_dataset, verbose=0)

  # Evaluate Model Size
  model_size = get_gzipped_model_size(model_path)

  # Evaluate Inference Time
  startTime = time.time()
  prediction = model.predict(val_dataset)
  executionTime = (time.time() - startTime)/len(val_dataset)

  # Print
  print('\nModel Accuracy:', test_acc*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Model Information": model_info,
                      "Accuracy": str(test_acc*100) + " %",
                      "Loss": str(test_loss*100) + " %",
                      "Model Size": str(float(model_size)/10**6) + " Mega bytes", # we want mega bytes
                      "Inference Time": str(executionTime) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()


  return test_acc, model_size, executionTime, evulation_df




# research paper evaluation function
def evaluate_research_paper_with_model_path(model_path):
  # model_path = "/content/models/solnet.hdf5"
  solnet = load_model(model_path, compile=False)
  history = solnet.history()
  plt.plot(history.history['loss'])
  plt.plot(history.history['acc'])
  plt.title('acc loss vs epoch')
  plt.xlabel('epoch')
  plt.legend(['loss', 'acc'], loc='upper left')
  plt.show()

def evaluate_research_paper_with_model(model):
  history = model.history()
  plt.plot(history.history['loss'])
  plt.plot(history.history['acc'])
  plt.title('acc loss vs epoch')
  plt.xlabel('epoch')
  plt.legend(['loss', 'acc'], loc='upper left')
  plt.show()

In [5]:
def iterative_pruning(model, initial_sparsity, final_sparsity, begin_step, end_step, tdata_loader,vdata_loader, epochs):
  prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

  # Define model for pruning.
  pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
          final_sparsity=final_sparsity, begin_step=begin_step, end_step=end_step, frequency=100)
  }

  pruned_model = prune_low_magnitude(model, **pruning_params)

  # `prune_low_magnitude` requires a recompile.
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
  pruned_model.compile(optimizer=Adam(.0001, .8, .9),
      loss=binary_crossentropy,
      metrics=['accuracy'])

  callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
  ]

  pruned_model.fit(tdata_loader, epochs=epochs, validation_data=vdata_loader, callbacks=callbacks)

  # Strip pruning wrappers
  stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

  return pruned_model, stripped_pruned_model

In [6]:
def print_model_weights_sparsity(model):

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            if "kernel" not in weight.name or "centroid" in weight.name:
                continue
            weight_size = weight.numpy().size
            zero_num = np.count_nonzero(weight == 0)
            print(
                f"{weight.name}: {zero_num/weight_size:.2%} sparsity ",
                f"({zero_num}/{weight_size})",
            )

# Load data

In [7]:
batch_size = 32
#location = 'dataset/'
location = "/content/drive/MyDrive/Solar_panel_dust_detection/dataset_1"
label_mode = 'binary'
seed = 10 #changed for each fold made manually

class_names = ['clean', 'dirty']
in_size = [227, 227, 3]

tr_dataset = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1], 
                                          subset = 'training', batch_size=batch_size, validation_split=.2)

val_dataset = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1],
                                          subset = 'validation', batch_size=batch_size, validation_split=.2)

Found 1440 files belonging to 2 classes.
Using 1152 files for training.
Found 1440 files belonging to 2 classes.
Using 288 files for validation.


# Load research paper model and Evaluate

In [9]:
research_paper_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/research_paper_model.h5'
research_paper_model = tf.keras.models.load_model(research_paper_model_path)

OSError: ignored

In [ ]:
research_model_test_acc, research_model_model_size, research_model_executionTime, research_model_evulation_df = evaluate_model_without_saving_stats(research_paper_model, research_paper_model_path, "#0", val_dataset)

In [ ]:
research_model_evulation_df

# Load base model and Evaluate


In [10]:
base_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/final_base_model.h5'
base_model = tf.keras.models.load_model(base_model_path)

In [11]:
base_model_test_acc, base_model_model_size, base_model_executionTime, base_model_evulation_df = evaluate_model_without_saving_stats(base_model, base_model_path, "#0", val_dataset)

9/9 [==============================] - 25s 39ms/step

Model Accuracy: 94.44444179534912 %
Model Size: 645039743.00 bytes
Inference Time is:  2.768649180730184 s


In [12]:
base_model_evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#0
2,Accuracy,94.44444179534912 %
3,Loss,17.469173669815063 %
4,Model Size,645.039743 Mega bytes
5,Inference Time,2.768649180730184 sec


# Load iteratively pruned model and Evaluate

In [ ]:
# Loading pruned models gives an error, hence we will iteratively prune it in the notebook

# Iteratively prune model and Evaluate

In [13]:
# frquency = 100
# target sparsity = 75%
# sparisty for each pruning tep = 75%/100 = 0.75%
# Total Number of Training Samples = 1440
# batch size = 32
# epochs = 10

# possible start and end for the iterative pruning schedule
# Number of learning steps = (1440/32)*10 = 450 (45, 400)
# Number of learning steps = (1440/32)*20 = 900 (90, 800)
# Number of learning steps = (1440/32)*25 = 1125 (110, 1010) -> chosen epoch of 25 epochs
# Number of learning steps = (1440/32)*30 = 1350 (135, 1200)
# Number of learning steps = (1440/32)*35 = 1575 (150, 1400)

# iterative_pruning(
                  #     model, 
                  #     initial_sparsity = 0, 
                  #     final_sparsity   = 0.7, 
                  #     begin_step       = 0, 110  (~10% of 5625) # pruning early on the pruning schedule to get optimal model size
                  #     end_step         = ?, 1010 (~80% of 5625) # pruning only till 80% so that model can still have enough leanring steps to learn and build networks for the data
                  #     train_images, 
                  #     train_labels, 
                  #     epochs           = 3, no change ? since the Number of Steps per Epoch = (Total Number of Training Samples = ?) / (Batch Size = ?) 
                  # )

iteratively_pruned_model, iteratively_pruned_stripped_model = iterative_pruning(base_model, 0, 0.75, 110, 1010, tr_dataset, val_dataset, 25)

Epoch 1/25
36/36 [==============================] - 383s 8s/step - loss: 1.0735 - accuracy: 0.7708 - val_loss: 4.8794 - val_accuracy: 0.5833
Epoch 2/25
36/36 [==============================] - 116s 3s/step - loss: 0.4543 - accuracy: 0.8976 - val_loss: 1.6374 - val_accuracy: 0.7326
Epoch 3/25
36/36 [==============================] - 117s 3s/step - loss: 0.4008 - accuracy: 0.9184 - val_loss: 1.9916 - val_accuracy: 0.6875
Epoch 4/25
36/36 [==============================] - 116s 3s/step - loss: 0.4560 - accuracy: 0.9123 - val_loss: 0.7575 - val_accuracy: 0.8194
Epoch 5/25
36/36 [==============================] - 137s 3s/step - loss: 0.2456 - accuracy: 0.9514 - val_loss: 9.5081 - val_accuracy: 0.5903
Epoch 6/25
36/36 [==============================] - 117s 3s/step - loss: 0.3647 - accuracy: 0.9306 - val_loss: 2.3454 - val_accuracy: 0.7257
Epoch 7/25
36/36 [==============================] - 121s 3s/step - loss: 0.2205 - accuracy: 0.9488 - val_loss: 3.7330 - val_accuracy: 0.6424
Epoch 8/25
36

In [14]:
iteratively_pruned_model_path = "iteratively_pruned_model.h5"
iteratively_pruned_stripped_model.save(iteratively_pruned_model_path)


iteratively_pruned_model.save('/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/iteratively_pruned_75_model.h5')
iteratively_pruned_stripped_model.save('/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/iteratively_stripped_pruned_75_model.h5')

In [15]:
pruned_test_acc, pruned_model_size, pruned_executionTime, pruned_evulation_df = evaluate_model_without_saving_stats(iteratively_pruned_model, iteratively_pruned_model_path, "#1", val_dataset)

9/9 [==============================] - 26s 37ms/step

Model Accuracy: 88.88888955116272 %
Model Size: 79142333.00 bytes
Inference Time is:  2.9017754660712347 s


In [16]:
print_model_weights_sparsity(iteratively_pruned_model)

conv2d/kernel:0: 74.18% sparsity  (25849/34848)
conv2d_1/kernel:0: 74.18% sparsity  (455743/614400)
conv2d_2/kernel:0: 74.18% sparsity  (656270/884736)
conv2d_3/kernel:0: 74.18% sparsity  (984405/1327104)
conv2d_4/kernel:0: 74.18% sparsity  (656270/884736)
dense/kernel:0: 74.18% sparsity  (28000862/37748736)
dense_1/kernel:0: 74.18% sparsity  (12444828/16777216)
dense_2/kernel:0: 74.17% sparsity  (3038/4096)


In [17]:
pruned_evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#1
2,Accuracy,88.88888955116272 %
3,Loss,46.273741126060486 %
4,Model Size,79.142333 Mega bytes
5,Inference Time,2.9017754660712347 sec


## Observation

We have almost 86% decrease in model size with minimal loss in accuracy.



| Base model | Iteratively pruned model|
|------------|------------|
| - Overall accuracy: 82.63%| - Overall accuracy: 82.63% (higher accuracy)|
| - Overall model size: 647.98 MB | - Overall model size: 89.66 MB (smaller size)|
| - Overall Inference time: 4.55 s | - Overall Inference time: 3.28 s (lower inference time)|
| - Performance: Lower| - Performance: Higher|
| - Key: It has a larger model size compared to iterative pruning,| - Key: Iterative pruning proves that we can fine tune our sparsity level, |
|  and significanlty has higher inference time.| while maintaining our accuracy and inference time.|

In [18]:
pruned_evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#1
2,Accuracy,88.88888955116272 %
3,Loss,46.273741126060486 %
4,Model Size,79.142333 Mega bytes
5,Inference Time,2.9017754660712347 sec


# Quantization

In [19]:
# A helper function to evaluate the TF Lite model using a test loader.
def evaluate_tflite_model(interpreter, model_path, test_loader):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the test dataset.
  prediction_digits = []
  accurate_count = 0
  total_images = 0
  total_time = 0

  for images, labels in val_dataset:
    total_images += len(images)
    for i in range(len(images)):
      # Pre-processing: add batch dimension and convert to float32 to match with
      # the model's input data format.
      test_image = np.expand_dims(images[i].numpy(), axis=0).astype(np.float32)
      interpreter.set_tensor(input_index, test_image)

      # Run inference.
      startTime = time.time()
      interpreter.invoke()
      executionTime = time.time() - startTime
      total_time += executionTime

      # Post-processing: remove batch dimension and find the digit with highest
      # probability.
      interpreter.invoke()
      output = interpreter.get_tensor(output_index)

      digit = np.round(output[0])
      prediction_digits.append(digit)

      # Compare prediction results with ground truth labels to calculate accuracy.
      if digit == labels[i].numpy():
        accurate_count += 1

  accuracy = accurate_count * 1.0 / total_images
  model_size = get_gzipped_model_size(model_path)

  # Print
  print('\nModel Accuracy:', accuracy*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Average Inference Time per Image: %.5f s" % (total_time/total_images))

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Accuracy": str(accuracy*100) + " %",
                      "Model Size": str(float(model_size)/10**6) + " Mega bytes", # we want mega bytes
                      "Average Inference Time per Image": str((total_time/total_images)) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()
  
  return evulation_df

## Base model

### 32 bit quanitization

In [20]:
converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float32]
fp32_quantized_tflite_base_model = converter.convert()

with open('fp32_quantized_tflite_base_model.tflite', 'wb') as f:
  f.write(fp32_quantized_tflite_base_model)
interpreter_fp32_quant_base_model = tf.lite.Interpreter(model_path=str('fp32_quantized_tflite_base_model.tflite'))
interpreter_fp32_quant_base_model.allocate_tensors()

In [21]:
quantized_32_bit_tflite_base_model_evulation_df = evaluate_tflite_model(interpreter_fp32_quant_base_model, 'fp32_quantized_tflite_base_model.tflite', val_dataset)
quantized_32_bit_tflite_base_model_evulation_df


Model Accuracy: 88.88888888888889 %
Model Size: 78731271.00 bytes
Average Inference Time per Image: 0.07895 s


,index,0
0,Evaluation type,Evualation
1,Accuracy,88.88888888888889 %
2,Model Size,78.731271 Mega bytes
3,Average Inference Time per Image,0.07895166012975904 sec


### 16 bit quanitization

In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
fp16_quantized_tflite_base_model = converter.convert()

with open('fp16_quantized_tflite_base_model.tflite', 'wb') as f:
  f.write(fp16_quantized_tflite_base_model)
interpreter_fp16_quant_base_model = tf.lite.Interpreter(model_path=str('fp16_quantized_tflite_base_model.tflite'))
interpreter_fp16_quant_base_model.allocate_tensors()

In [23]:
quantized_16_bit_tflite_base_model_evulation_df = evaluate_tflite_model(interpreter_fp16_quant_base_model, 'fp16_quantized_tflite_base_model.tflite', val_dataset)
quantized_16_bit_tflite_base_model_evulation_df


Model Accuracy: 88.88888888888889 %
Model Size: 44957703.00 bytes
Average Inference Time per Image: 0.07796 s


,index,0
0,Evaluation type,Evualation
1,Accuracy,88.88888888888889 %
2,Model Size,44.957703 Mega bytes
3,Average Inference Time per Image,0.07795967078871197 sec


### 8 bit quanitization

In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_types = [tf.float16]
fp8_quantized_tflite_base_model = converter.convert()

with open('fp8_quantized_tflite_base_model.tflite', 'wb') as f:
  f.write(fp8_quantized_tflite_base_model)
interpreter_fp8_quant_base_model = tf.lite.Interpreter(model_path=str('fp8_quantized_tflite_base_model.tflite'))
interpreter_fp8_quant_base_model.allocate_tensors()

In [25]:
quantized_8_bit_tflite_base_model_evulation_df = evaluate_tflite_model(interpreter_fp8_quant_base_model, 'fp8_quantized_tflite_base_model.tflite', val_dataset)
quantized_8_bit_tflite_base_model_evulation_df


Model Accuracy: 89.93055555555556 %
Model Size: 22582982.00 bytes
Average Inference Time per Image: 0.06260 s


,index,0
0,Evaluation type,Evualation
1,Accuracy,89.93055555555556 %
2,Model Size,22.582982 Mega bytes
3,Average Inference Time per Image,0.06260265823867586 sec


## Pruned model

### 32 bit quantization

In [26]:
converter = tf.lite.TFLiteConverter.from_keras_model(iteratively_pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float32]
fp16_quantized_tflite_iteratively_pruned_model = converter.convert()

with open('fp32_quantized_tflite_iteratively_pruned_model.tflite', 'wb') as f:
  f.write(fp16_quantized_tflite_iteratively_pruned_model)
interpreter_fp32_quant_iteratively_pruned_model = tf.lite.Interpreter(model_path=str('fp32_quantized_tflite_iteratively_pruned_model.tflite'))
interpreter_fp32_quant_iteratively_pruned_model.allocate_tensors()

In [27]:
quantized_32_bit_tflite_iteratively_pruned_model_evulation_df = evaluate_tflite_model(interpreter_fp32_quant_iteratively_pruned_model, 'fp32_quantized_tflite_iteratively_pruned_model.tflite', val_dataset)
quantized_32_bit_tflite_iteratively_pruned_model_evulation_df


Model Accuracy: 88.88888888888889 %
Model Size: 91485732.00 bytes
Average Inference Time per Image: 0.40434 s


,index,0
0,Evaluation type,Evualation
1,Accuracy,88.88888888888889 %
2,Model Size,91.485732 Mega bytes
3,Average Inference Time per Image,0.4043370758493741 sec


### 16 bit quantization

In [28]:
converter = tf.lite.TFLiteConverter.from_keras_model(iteratively_pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
fp16_quantized_tflite_iteratively_pruned_model = converter.convert()

with open('fp16_quantized_tflite_iteratively_pruned_model.tflite', 'wb') as f:
  f.write(fp16_quantized_tflite_iteratively_pruned_model)
interpreter_fp16_quant_iteratively_pruned_model = tf.lite.Interpreter(model_path=str('fp16_quantized_tflite_iteratively_pruned_model.tflite'))
interpreter_fp16_quant_iteratively_pruned_model.allocate_tensors()

In [29]:
quantized_16_bit_tflite_iteratively_pruned_model_evulation_df = evaluate_tflite_model(interpreter_fp16_quant_iteratively_pruned_model, 'fp16_quantized_tflite_iteratively_pruned_model.tflite', val_dataset)
quantized_16_bit_tflite_iteratively_pruned_model_evulation_df


Model Accuracy: 88.88888888888889 %
Model Size: 55500584.00 bytes
Average Inference Time per Image: 0.40406 s


,index,0
0,Evaluation type,Evualation
1,Accuracy,88.88888888888889 %
2,Model Size,55.500584 Mega bytes
3,Average Inference Time per Image,0.4040607288479805 sec


### 8 bit quantization

In [30]:
converter = tf.lite.TFLiteConverter.from_keras_model(iteratively_pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_types = [tf.float16]
fp8_quantized_tflite_iteratively_pruned_model = converter.convert()

with open('fp8_quantized_tflite_iteratively_pruned_model.tflite', 'wb') as f:
  f.write(fp8_quantized_tflite_iteratively_pruned_model)
interpreter_fp8_quant_iteratively_pruned_model = tf.lite.Interpreter(model_path=str('fp8_quantized_tflite_iteratively_pruned_model.tflite'))
interpreter_fp8_quant_iteratively_pruned_model.allocate_tensors()

In [31]:
quantized_8_bit_tflite_iteratively_pruned_model_evulation_df = evaluate_tflite_model(interpreter_fp8_quant_iteratively_pruned_model, 'fp8_quantized_tflite_iteratively_pruned_model.tflite', val_dataset)
quantized_8_bit_tflite_iteratively_pruned_model_evulation_df


Model Accuracy: 88.88888888888889 %
Model Size: 91485750.00 bytes
Average Inference Time per Image: 0.42835 s


,index,0
0,Evaluation type,Evualation
1,Accuracy,88.88888888888889 %
2,Model Size,91.48575 Mega bytes
3,Average Inference Time per Image,0.42834601054588956 sec


## Observation

